 # Przykładowy problem 1

 Witaj. Tu znajdziesz opis przykładowego problemu dotyczącego rekomendacji (być może pośrednio reguł asocjacyjnych, a może masz inny pomysł?). Problem stanowi implementacja systemu oceniającego. Ten notebook pokaże w jaki sposób wykonywana jest analiza jakości systemu oceniającego. Zapoznajmy się z poszczególnymi elementami systemu.

 ### Użytkownicy testowi

  Na ich podstawie wyznaczana będzie jakość systemu oceniającego. Ocen wystawionych przez użytkowników testowych BEZWZGLĘDNIE NIE MOŻNA BRAĆ POD UWAGĘ podczas tworzenia systemu. Użytkownicy testowi, to po prostu losowo wybrani użytkownicy spośród wszystkich użytkowników. Ten losowy wybór znajduje się w pliku test_users.py - jego treści nie powinieneś zmieniać.



In [110]:
import test_users
import numpy as np
import csv
from tqdm import tqdm

test_users = test_users.test_users

 ### Filmy i użytkownicy

 Poniższe klasy reprezentują koncepty filmu i użytkownika. Zauważ, że struktury danych ograniczają się do prostych identyfikatorów filmów i ocen wystawionych przez użytkowników, dodatkowo dla obydwóch typów tworzone są indeksy. Ta implementacja **nie** może ulec zmianie. Jeżeli chcesz mieć dostęp do innych danych (np. do gatunków filmów, albo do opisów niepochodzących ze źródła danych), umieść implementacje źródeł danych w swoim systemie rekomendacyjnym, a nie tutaj.

In [35]:
class Movie:
    index = {}
    name_index = {}
    inner_index = {}
    reverse_inner_index = {}
    inner_index_gen = 0
    def __init__(self, id, name):
        self.id = id
        self.name = name
        self.ratings = []
        self.genres = []
        Movie.index[id] = self
        Movie.name_index[name] = self
    def add_rating(self, rating):
        self.ratings.append(rating)
    
    
class User:
    index = {}
    def __init__(self, id):
        self.id = id
        self.ratings = {}
        User.index[id] = self
    def add_rating(self, movie, rating):
        movie.add_rating(rating)
        self.ratings[movie.id] = rating
    def __str__(self):
        str_bldr = f'{self.id}'
        return str_bldr

        


### Indeksy

Indeksy danych wypełniane są danymi z plików.


In [36]:

with open('data/movie.csv', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    csv_reader.__next__()
    for line in csv_reader:
        Movie(int(line[0]), line[1])

with open('data/rating.csv', encoding='utf-8') as file:
    csv_reader = csv.reader(file)
    csv_reader.__next__()
    for line in tqdm(csv_reader, total=20000263):
        if not int(line[0]) in User.index.keys():
            User(int(line[0]))
        User.index[int(line[0])].add_rating(Movie.index[int(line[1])],float(line[2]))

100%|██████████| 20000263/20000263 [00:20<00:00, 960519.60it/s] 


 ### Systemy oceniające - klasa bazowa

 Każdy system oceniający ma dostęp do wszystkich użytkowników i ocen, które nie są ocenami użytkowników testowych. Można również zaimplementować metody doboru innych danych (np. gatunków, tagów, danych zewnętrznych). Poniżej znajduje się klasa bazowa - w jej inicjalizatorze wczytywane są dotyczące ocen wystawionych przez użytkowników nie będących testowymi.

In [ ]:
class RatingSystem:
    def __init__(self):
        self.users = {id : User.index[id] for id in User.index if id not in test_users}
        self.movie_ratings = {}
        for user in tqdm(self.users):
            for movie in self.users[user].ratings:
                if movie not in self.movie_ratings.keys():
                    self.movie_ratings[movie] = [self.users[user].ratings[movie]]
                else:
                    self.movie_ratings[movie].append(self.users[user].ratings[movie])
        
    def rate(self, user, movie):
        return


 ### Przykłady prostych systemów oceniających

 Poniższe przykłady są proste. Nie wykorzystują efektywnych rozwiązań (np. biblioteki numpy do obliczeń). Hipotezy, które te systemy realizują są bardzo naiwne. Twoim zadaniem jest napisanie systemu oceniającego, który będzie lepszy od tych systemów. Twój system będzie również porównany z systemami kolegów z grupy. Im wyższa pozycja w rankingu tym więcej punktów można uzyskać. Jeżeli zaimplementowany przez Ciebie system będzie lepszy niż system 'Naive Rating' otrzymasz 5 punktów; lepszy niż 'Average Global Movie Rating' - 7 punktów; lepszy niż wszystkie 4 przykładowe systemy - 10 punktów. Dodatkowo, jeżeli Twój system będzie jednym z najlepszych wśród implementacji kolegów z roku to otrzymasz:

 1. miejsce - 25 punktów
 2. miejsce - 20 punktów 
 3. miejsce - 15 punktów

 Przy czym, w przypadku wielu implementacji dających takie same wyniki, żeby otrzymać punkty za miejsce, system musi być lepszy, niż 6 najlepszy system (nie może remisować z systemem na pozycji 6).  
 
 Implementując swój system oceniający użyj następującej funkcji to string:

```
     def __str__(self):
        return 'Dowolna nazwa (MOJ NUMER INDEKSU)'
```

In [120]:
class NaiveRating(RatingSystem):
    def __init__(self):
        super().__init__()
    def rate(self, user, movie):
        return 2.5
    def __str__(self):
        return 'Naive Rating'

class AverageMovieRating(RatingSystem):
    def __init__(self):
        super().__init__()
    def rate(self, user, movie):
        n = len(self.movie_ratings[movie])
        if n == 0:
            return 2.5
        else:
            return sum(self.movie_ratings[movie])/n
    def __str__(self):
        return 'Average Movie Rating'
class AverageUserRating(RatingSystem):
    def __init__(self):
        super().__init__()
    def rate(self, user, movie):
        n = len(user.ratings.values())
        if n == 0:
            return 2.5
        else:
            return sum(user.ratings.values())/n
    def __str__(self):
        return 'Average User Rating'

class GlobalAverageMovieRating(RatingSystem):
    def __init__(self):
        super().__init__()
        self.GlobalAverageMovieRating = 0
        self.TotalMovies = 0
        for movie in self.movie_ratings:
            for rating  in self.movie_ratings[movie]:
                self.GlobalAverageMovieRating += rating
                self.TotalMovies += 1
        self.GlobalAverageMovieRating /= self.TotalMovies

    def rate(self, user, movie):
        return self.GlobalAverageMovieRating
    def __str__(self):
        return 'Average Global Movie Rating'
    
class Cheater(RatingSystem):
    def __init__(self):
        super().__init__()

    def rate(self, user, movie):
        if movie in user.ratings:
            return user.ratings[movie]
        else:
            return 2.5
    def __str__(self):
        return 'Cheater'


 ### Ocena systemów

 Poniższe klasy dotyczą oceny systemów - zwróć uwagę na parametr verbose, służy on do ograniczania informacji zwrotnej

In [123]:
import copy
class RatingSystemCompetition:
    
    def __init__(self):
        self.registered_systems = []
        self.users = {id : User.index[id] for id in User.index if id not in test_users}
        self.verbose = 2
    def register(self, system):
        self.registered_systems.append(system)
        
    def build_round_robin(self):
        self.pairs = {}
        for system in self.registered_systems:
            self.pairs[system]  = []
            for competitor in self.registered_systems:
                if str(system) != str(competitor):
                    self.pairs[system].append((system, competitor))

            
    def runMatch(self, system, competitor):
        users_ids = np.random.choice(np.array(list(self.users.keys())), size=100)
        score = 0
        wins = 0
        loses = 0
        draws = 0
        for user_id in users_ids:
            user = self.users[user_id]
            user_copy = copy.deepcopy(self.users[user_id])
            movie_id = np.random.choice(np.array(list(user.ratings.keys())), size=1)[0]
            del user_copy.ratings[movie_id]
            true_rating = self.users[user_id].ratings[movie_id]
            system_rating = system.rate(user_copy,movie_id)
            competitor_rating = competitor.rate(user_copy,movie_id)
            
            if abs(true_rating - system_rating) <  abs(true_rating - competitor_rating):
                score += 1
                wins += 1
            elif abs(true_rating - system_rating) >  abs(true_rating - competitor_rating):
                score -= 1
                loses += 1
            else:
                draws += 1
                
        return score, wins, draws, loses
    
    def compete(self):
        self.total_scores = {}
        for system in self.pairs:
            self.total_scores[system] = 0
            if self.verbose >= 2: print(f'{system} analysis: ')
            for matchup in self.pairs[system]:
                score, wins, draws, loses = self.runMatch(matchup[0],matchup[1])
                if self.verbose >= 2: print(f'{matchup[0]} vs {matchup[1]} : {score} ({wins} wins, {draws} draws, {loses} loses)')
                self.total_scores[system] += score
            if self.verbose >= 2: print(f'{system} score: {self.total_scores[system]}')
        if self.verbose >= 1:
            print('Final scores: ')
            place = 1
            for system in sorted(self.total_scores, key=self.total_scores.get, reverse=True):
                print(f'{place}. {system}, {self.total_scores[system]} pkt')
                place += 1
            
        
            
            
            
            
    

 ### Przykładowa analiza

In [124]:
competition = RatingSystemCompetition()
competition.register(GlobalAverageMovieRating())
competition.register(NaiveRating())
competition.register(AverageMovieRating())
competition.register(AverageUserRating())
competition.register(Cheater())
competition.build_round_robin()
competition.compete()

100%|██████████| 137495/137495 [00:01<00:00, 70676.25it/s]


Average Global Movie Rating analysis: 
Average Global Movie Rating vs Naive Rating : 34 (67 wins, 0 draws, 33 loses)
Average Global Movie Rating vs Average Movie Rating : -8 (46 wins, 0 draws, 54 loses)
Average Global Movie Rating vs Average User Rating : -18 (41 wins, 0 draws, 59 loses)
Average Global Movie Rating vs Cheater : 30 (65 wins, 0 draws, 35 loses)
Average Global Movie Rating score: 38
Naive Rating analysis: 
Naive Rating vs Average Global Movie Rating : -26 (37 wins, 0 draws, 63 loses)
Naive Rating vs Average Movie Rating : -58 (21 wins, 0 draws, 79 loses)
Naive Rating vs Average User Rating : -53 (23 wins, 1 draws, 76 loses)
Naive Rating vs Cheater : 0 (0 wins, 100 draws, 0 loses)
Naive Rating score: -137
Average Movie Rating analysis: 
Average Movie Rating vs Average Global Movie Rating : 44 (72 wins, 0 draws, 28 loses)
Average Movie Rating vs Naive Rating : 54 (77 wins, 0 draws, 23 loses)
Average Movie Rating vs Average User Rating : -2 (49 wins, 0 draws, 51 loses)
Avera